In [2]:
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "C:\Users\MINHYE\anaconda3\envs\project\python.exe"
  * The NumPy version is: "1.21.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: 지정된 모듈을 찾을 수 없습니다.


In [32]:
df_on = pd.read_csv('day_on.csv')

In [33]:
# 너무 큰 데이터 > 역별 승차 인원 평균 2만 이상 역들만 취합
mean_data =  df_on['total'].groupby(df_on['st_num']).mean().reset_index()
f_st = mean_data[mean_data.total > 20000]['st_num']

df=pd.DataFrame()
for f in f_st:
  df = pd.concat([df,df_on.loc[df_on.st_num == f]], ignore_index=True)
df.to_csv('scaled_on')

In [34]:
test = df[['total', 'avgTa', 'minTa', 'maxTa', 'sumRn', 'day_info', 'bus_cnt']]
t =test.drop_duplicates().reset_index(drop=True)

features = ['avgTa', 'minTa', 'maxTa', 'sumRn', 'day_info', 'bus_cnt']
target = 'total'

t_y = t[target]
t_x = t[features]

from sklearn.model_selection import train_test_split

t_x_train, t_x_test, t_y_train, t_y_test = train_test_split(t_x, t_y, train_size=0.80, test_size=0.20, random_state=42)

t_x_train.shape, t_x_test.shape, t_y_train.shape, t_y_test.shape

((12291, 6), (3073, 6), (12291,), (3073,))

In [1]:
pipe = make_pipeline( 
    OrdinalEncoder(),
    SimpleImputer(), 
    StandardScaler(), 
    RandomForestRegressor()
)
pipe.fit(t_x_train, t_y_train)
y_pred = pipe.predict(t_x_test)

NameError: name 'make_pipeline' is not defined

In [36]:
mse = mean_squared_error(t_y_test, y_pred)
mae = mean_absolute_error(t_y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(t_y_test, y_pred)

res = pd.DataFrame([['MSE', mse],['MAE', mae],['RMSE', rmse],['R2', r2]], columns=['Metric', 'Score'])
res

,Metric,Score
0,MSE,1.663083e+08
1,MAE,8.907516e+03
2,RMSE,1.289606e+04
3,R2,3.488386e-01
